In [2]:
includet(joinpath(ENV["JULIA_DQMC"], "src/measure_framework.jl"))

In [5]:
measxml = "dqmc.meas.xml"
mp = measxml2MeasParams(measxml)

Loading DQMC parameters (.in.xml)
L = 4
SLICES = 100
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 100
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3


MeasParams
  requested: Set{Symbol}
  todo: Set{Symbol}
  todo_inonego: Set{Symbol}
  todo_insteps: Set{Symbol}
  meas_infile: String "dqmc.meas.xml"
  outfile: String "dqmc.meas.h5"
  p: Params
  dqmc_outfile: String "dqmc.out.h5"
  overwrite: Bool true
  num_threads: Int64 1
  walltimelimit: DateTime
  save_after: Int64 2
  to: TimerOutput
  confs_iterator_start: Int64 1


In [6]:
mp.requested

Set(Symbol[:etpc_minus, :etcdc_plus, :chi_dyn_symm, :etpc_plus, :etcdc_minus, :chi_dyn])

In [686]:
#obs_inonego, obs_insteps = measure(mp, debug = true);

In [687]:
#mp.walltimelimit = now() + Second(5)

2019-05-07T15:28:01.379

In [ ]:
measure(mp)

mp.todo_inonego = Set(Symbol[:chi_dyn_symm, :chi_dyn])
mp.todo_insteps = Set(Symbol[:etpc_minus, :etcdc_plus, :etpc_plus, :etcdc_minus])


In [ ]:
mp.

In [676]:
            titer = mp.to["iteration"]
            iterdur = TimerOutputs.time(titer) *10.0^(-9)/TimerOutputs.ncalls(titer)

0.06654364020000006

In [3]:
jldopen("dqmc.meas.h5", "r") do f
    global Pminus = read(f["obj/etpc_minus"])
end

LightObservable{Array{Float64,2},3}
| Name: etpc_minus
| Measurements: 2

In [4]:
Pminus

LightObservable{Array{Float64,2},3}
| Name: etpc_minus
| Measurements: 2

In [164]:
  measure(mp, obs, confs, greens, mc)

Measuring ...
1
2
3
4
5


 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            940ms / 2.42%           1.80MiB / 90.3%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 measure loop               1   22.8ms   100%  22.8ms   1.63MiB  100%   1.63MiB
   load etgf                5   8.59ms  37.7%  1.72ms    428KiB  25.7%  85.6KiB
   bosonic                  5   1.30ms  5.73%   261μs   0.96MiB  59.2%   197KiB
     chi                    5   1.19ms  5.25%   239μs   0.96MiB  58.9%   196KiB
     binder                 5   96.5μs  0.42%  19.3μs   1.95KiB  0.12%     400B
   fermionic                5    252μs  1.11%  50.5μs    200KiB  12.0%  40.0KiB
     etpc                   5    163μs 

In [235]:
  export_results(mp, obs, nsweeps)

Calculating errors and exporting...
Done. Exported to dqmc.meas.h5.


In [298]:
@macroexpand @addobs asd Float64

quote
    #= C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\measure_framework.jl:84 =#
    if (Main.Symbol)("asd") in mp.todo_insteps
        #= C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\measure_framework.jl:85 =#
        obs_insteps = (Main.add)(obs_insteps, asd=(Main.create_or_load_obs)("asd", Float64, name="asd", alloc=num_confs))
    else
        #= C:\Users\carsten\Desktop\sciebo\codes\julia-sdw-dqmc\src\measure_framework.jl:87 =#
        obs_inonego = (Main.add)(obs_inonego, asd=(Main.Observable)(Float64, name="asd", alloc=num_confs))
    end
end

In [33]:
typeof(esc(:asd))

Expr

In [40]:
:(:)

:(:)

In [41]:
typeof(ans)

Symbol

In [88]:
x = :whatever
colon = :(:)

:(:)

In [91]:
:($x)

:whatever

In [116]:
eval(:($(Expr(:carsten))))

ErrorException: syntax: invalid syntax (carsten)

In [113]:
eval(:(3+3))

6

In [239]:
macro iswhatever(x::Symbol)
    return quote 
        $(Expr(x))
    end
end

@print_given_symbol (macro with 1 method)

In [240]:
@macroexpand @print_given_symbol d

x = :d


quote
    #= In[239]:4 =#
    $(Expr(:d))
end

In [242]:
@print_given_symbol d

x = :d


ErrorException: syntax: invalid syntax (d)

In [250]:
Meta.parse(":d")

:(:d)

In [252]:
Meta.quot(Expr(:$, :(1+2)))

TypeError: TypeError: in Expr, expected Symbol, got Colon

In [253]:
@which @show x

@show(__source__::LineNumberNode, __module__::Module, exs...) in Base at show.jl:552

In [325]:
TIMING

false

In [358]:
@code_lowered measure_fermionic(mp, obs_inonego, conf, mc.s.greens, mc, 1)

CodeInfo(
1 ─ %1  = (Main.isnothing)(mc)
│   %2  = !%1
└──       goto #6 if not %2
2 ─ %4  = (Base.getproperty)(mc, :p)
│   %5  = Base.setproperty! === Core.setfield!
└──       goto #4 if not %5
3 ─ %7  = (Core.typeof)(%4)
│   %8  = (Core.fieldtype)(%7, :hsfield)
│         #temp# = (Base.convert)(%8, conf)
└──       goto #5
4 ─       #temp# = conf
5 ┄ %12 = #temp#
└──       (Base.setproperty!)(%4, :hsfield, %12)
6 ┄ %14 = :etpc_plus in $(Expr(:static_parameter, 1))
└──       goto #8 if not %14
7 ─       (Main.measure_etpc!)(mc, greens)
│   %17 = (Base.getindex)(obs, :etpc_plus)
│   %18 = (Base.getproperty)(mc, :s)
│   %19 = (Base.getproperty)(%18, :meas)
│   %20 = (Base.getproperty)(%19, :etpc_plus)
│         (Main.push!)(%17, %20)
│   %22 = (Base.getindex)(obs, :etpc_minus)
│   %23 = (Base.getproperty)(mc, :s)
│   %24 = (Base.getproperty)(%23, :meas)
│   %25 = (Base.getproperty)(%24, :etpc_minus)
└──       (Main.push!)(%22, %25)
8 ┄       return Main.nothing
)

In [335]:
nt = (a="test", b=5)

(a = "test", b = 5)

In [355]:
function f(nt::NamedTuple{K,V}) where {K,V}
    return :d in K
end

f (generic function with 1 method)

In [357]:
@code_lowered f(nt)

CodeInfo(
1 ─ %1 = :d in $(Expr(:static_parameter, 1))
└──      return %1
)